## Тестовое задание 1

**Содержание**:

0. Оригинальное решение
1. Проверка на сгенерированных данных чуть большим объёмом

### 0. Оригинальное решение

In [1]:
import pandas as pd
import psycopg2
import os

**Общая идея**:
1) Запускаем запрос с входными параметрами\
2) Из выгруженного массива собираем все уникальные значения из разных столбцов в соответствующие кортежи\
3) После делаем второй запрос с перебором всех возможных вариантов из кортежей, и так далее...\
4) После каждой итерации сохраняем число строк в выгруженном массиве. Когда число строк перестает увеличиваться, выводим результат.

In [2]:
def task1_func(id=-1, phone=-1, mail='??'):
    
    #создаем подключение к бд
    dbpassword = os.getenv('pgAdmin_password')
    engine = psycopg2.connect(host='localhost', dbname='SBER_test', user='postgres', password=dbpassword)

    #значения аргументов функции по умолчанию и первых элементов множеств нужны, чтобы кортежи, которые мы вставляем\
    #в запрос, по виду не отличались от синтаксиса SQL: чтобы сразу было (a,b), а не (a,) например, а то sql ругается
    #задаем начальные множества по каждому параметру
    id_set = {-2,id}
    phone_set = {-2,phone}
    mail_set = {'?',mail}
    #также задаем два значения для списка с количеством строк в результате запроса, чтобы цикл запустился\ 
    #без лишних условий
    result_nrows = [-1,0]
    
    while result_nrows[-2] != result_nrows[-1]:
        #формируем запрос с проверкой на входимость значений по столбцам в наши множества
        query = f'''select * from task1
        where id in {tuple(id_set)} or
              phone in {tuple(phone_set)} or
              mail in {tuple(mail_set)}'''
        #выводим результат по запросу
        result = pd.read_sql_query(query,con=engine)
        #добавляем кол-во строк в выведенном массиве в список
        result_nrows.append(result.shape[0])
        #добавляем новые данные в множества вариантов и повторяем алгоритм
        id_set.update(result.id)
        phone_set.update(result.phone)
        mail_set.update(result.mail)
    #если по запросу данные не найдены, сообщаем об этом
    if result_nrows[-1] == 0:
        return 'No results, change input'
    else:
        return result

In [3]:
task1_func(phone = 87778885566)

,key,id,phone,mail
0,2,54321,87778885566,two@mail.ru
1,4,66678,87778885566,four@mail.ru
2,5,34567,84547895566,four@mail.ru
3,6,34567,89087545678,five@mail.ru


In [4]:
task1_func(mail = 'two@mail.ru')

,key,id,phone,mail
0,2,54321,87778885566,two@mail.ru
1,4,66678,87778885566,four@mail.ru
2,5,34567,84547895566,four@mail.ru
3,6,34567,89087545678,five@mail.ru


In [5]:
task1_func(mail = 'two@ya.ru')

'No results, change input'

### 1. Проверка на сгенерированных данных чуть большим объёмом

Импортируем тестовые данные напрямую в ноутбук без транзита через sql, чтобы не усложнять. 

In [6]:
test = pd.read_excel('~\\test.xlsx').drop(columns='Unnamed: 0')
test.head()

,key,id,phone,mail
0,0,89001,84758280647,fcormbgnvgwxyrxrndfzsnjb@mail.ru
1,1,78010,83572782012,rjmrvewiqzmcjqjbfqbxjztklzojyiw@mail.ru
2,2,97883,82971429310,alqinkfjqtqdruoczxyxrjjjzxsjxmr@mail.ru
3,3,36435,88388678263,aaiv@mail.ru
4,4,93636,81130830484,jqgonuwprhx@mail.ru


In [7]:
test2 = pd.read_excel('~\\test2.xlsx').drop(columns='Unnamed: 0')
test2.head()

,key,id,phone,mail
0,0,674448,82711735625,wxlqcywqdhaaheortlpdb@mail.ru
1,1,353974,85375805342,myikngkjmxqezoxclysteuoubgwsofjgkr@mail.ru
2,2,735332,81134824477,xuiwkdyurhilevrosyhxfhqsqqqndjqc@mail.ru
3,3,118908,88761347200,imxcvfojmrowermtdyjxwgcr@mail.ru
4,4,948702,83294687381,jiaiulghdishvbqcfp@mail.ru


Удалим из оригинальной функции чтение данных из sql, добавим параметр имени датасета (data).

In [8]:
def task1_func(data, id=-1, phone=-1, mail='??'):
    #значения аргументов функции по умолчанию и первых элементов множеств нужны, чтобы кортежи, которые мы вставляем\
    #в запрос, по виду не отличались от синтаксиса SQL: чтобы сразу было (a,b), а не (a,) например, а то sql ругается
    #задаем начальные множества по каждому параметру
    id_set = {id}
    phone_set = {phone}
    mail_set = {mail}
    #также задаем два значения для списка с количеством строк в результате запроса, чтобы цикл запустился\ 
    #без лишних условий
    result_nrows = [-1,0]
    
    while result_nrows[-2] != result_nrows[-1]:
        #выводим результат по запросу
        result = data[(data.id.isin(id_set)) | \
                       (data.phone.isin(phone_set)) | \
                       (data.mail.isin(mail_set))]
        #добавляем кол-во строк в выведенном массиве в список
        result_nrows.append(result.shape[0])
        #добавляем новые данные в множества вариантов и повторяем алгоритм
        id_set.update(result.id)
        phone_set.update(result.phone)
        mail_set.update(result.mail)
    #если по запросу данные не найдены, сообщаем об этом
    if result_nrows[-1] == 0:
        return 'No results, change input'
    else:
        return result

**Проверяем данные по датасету test2 для id 3,4**.

In [9]:
task1_func(test2, id=3)

,key,id,phone,mail
156721,156721,3,82198528995,tghqclyykngmzhjcqqvbbltevkl@mail.ru


In [10]:
task1_func(test2, id=4)

,key,id,phone,mail
339064,339064,4,85514996363,gjjzfhxnmqezafmuojloiabpdzhjkbi@mail.ru


Найдем "связанные" строки по запросам прямой фильтрацией, чтобы убедиться в корректности полученных результатов.

Для id = 3:

In [11]:
test2.query('id == 3')

,key,id,phone,mail
156721,156721,3,82198528995,tghqclyykngmzhjcqqvbbltevkl@mail.ru


In [12]:
test2.query('id == 3 | phone == 82198528995 | mail == "tghqclyykngmzhjcqqvbbltevkl@mail.ru"')

,key,id,phone,mail
156721,156721,3,82198528995,tghqclyykngmzhjcqqvbbltevkl@mail.ru


Для id = 4:

In [13]:
test2.query('id == 4')

,key,id,phone,mail
339064,339064,4,85514996363,gjjzfhxnmqezafmuojloiabpdzhjkbi@mail.ru


In [14]:
test2.query('id == 4 | phone == 85514996363	 | mail == "gjjzfhxnmqezafmuojloiabpdzhjkbi@mail.ru"')

,key,id,phone,mail
339064,339064,4,85514996363,gjjzfhxnmqezafmuojloiabpdzhjkbi@mail.ru


Результаты корректны, все параметры по этим id уникальны.

Более показательный результат по этому датасету можно получить по запросу phone = 89723802783.

In [15]:
task1_func(test2, phone=89723802783)

,key,id,phone,mail
280575,280575,575695,89723802783,zjhbefgnulnqrkuzzcgtuybffnguibfqme@mail.ru
294613,294613,514455,89723802783,taudkywbceusaavgziwidphetqlbs@mail.ru
464170,464170,514455,85503677037,uinrieggnjgzxx@mail.ru


Перейдем ко второму датасету.

**Проверяем данные по файлу test для id 22296, 35026, 54119**.

In [16]:
task1_func(test, id=22296)

,key,id,phone,mail
8068,8068,16137,86350580710,twrnepja@mail.ru
28326,28326,22296,82750671293,jfcivwlghrampynyexio@mail.ru
58141,58141,36057,80775677213,krgxswekentolramxxwyjvymwyiqtbfbln@mail.ru
111729,111729,22296,87875724457,rpdbkihbzxebilouiwuomzfle@mail.ru
130000,130000,22296,88171207757,fsldrjgam@mail.ru
135531,135531,22296,81787715958,wivq@mail.ru
202906,202906,22296,86393323637,fmhgxo@mail.ru
209486,209486,36057,80654020342,ndqfguqwdaersxeepjswbglvjgwqccscxvr@mail.ru
282349,282349,22296,82305242696,eiyxknwueikmyeiunpzskuvd@mail.ru
291893,291893,61570,84112805432,qsqanznsfykicgnqzycadfozdkntxfigjq@mail.ru


In [17]:
task1_func(test, id=35026)

,key,id,phone,mail
9249,9249,35026,88859961198,erolvlycajfgqigosbcrmuoq@mail.ru
14670,14670,80968,83951606408,lsdlhnzzjubio@mail.ru
15428,15428,35026,83668041547,tqsbraqgufg@mail.ru
130433,130433,35026,80461061757,ilaz@mail.ru
174832,174832,80968,87588047659,nokm@mail.ru
200403,200403,63408,88619533449,qqfivktiprbntyro@mail.ru
210709,210709,35026,88985682389,icgmasfjdancjvegxklsnfprdfnfxlv@mail.ru
213951,213951,35026,83978535869,gjuocbhuepsfwiueizrakdpdwmonrptzrf@mail.ru
246225,246225,80968,86802329400,zuyst@mail.ru
268282,268282,72130,81504117095,oigmvppsfvnqqfwmzopackjgqnzlnvxh@mail.ru


In [18]:
task1_func(test, id=54119)

,key,id,phone,mail
5194,5194,54119,89552293548,kxjpfbq@mail.ru
19182,19182,54119,82821279200,bhlidahdqanusraft@mail.ru
29727,29727,54119,83034267744,wlbiitnjiwqatatqfqy@mail.ru
34343,34343,99640,87256090916,vrkffzlrawwi@mail.ru
159487,159487,36444,84477973548,nrdqclrrlxsmlmsep@mail.ru
198071,198071,54119,80613616244,fdjz@mail.ru
198694,198694,99640,87445907887,xgpjvfvzvgclahzaqbepqtbby@mail.ru
219484,219484,54119,82661711478,dswfy@mail.ru
221317,221317,99640,83837257718,gldrzeiulsq@mail.ru
324176,324176,54119,85930643579,ohnckhb@mail.ru


Получили следующие результаты.

In [19]:
id = [22296, 35026, 54119]
res = pd.DataFrame()
for i in range(len(id)):
    res = pd.concat([res, pd.DataFrame({'id':[id[i]], 'Количество строк': [task1_func(test, id=id[i]).shape[0]]})])
res

,id,Количество строк
0,22296,46
0,35026,39
0,54119,35
